## Data Modelling

#### Required fields
* Transaction type (i.e. sale vs. rent - string)
* Bedrooms (integer)
* Bathrooms (integer)
* Description (free text string)
* Property type e.g. flat, detached house, terraced house
* Price e.g. 500,000 (typically integer)
* Location :  Key location data here is Postcode district and/or Postcode
* Agent (advertising the property)
* Listing source
* Listing URL
* Other nice-to-have metadata
* If a rental property is furnished or not
* Anything else you deem interesting

In [96]:
# import libaries needed
import pandas as pd
import psycopg2
from datetime import datetime, date

In [97]:
# load dataset
rm_data = pd.read_csv(f'data_output/rightnow_{date.today()}.csv')
omt_data = pd.read_csv(f'data_output/omt_{date.today()}.csv')
z_data = pd.read_csv(f'data_output/zoopla_{date.today()}.csv')

In [98]:
rm_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   transaction     50 non-null     object 
 1   address         50 non-null     object 
 2   bedroom         48 non-null     float64
 3   bathroom        49 non-null     float64
 4   sales_price     25 non-null     float64
 5   rent_perMonth   25 non-null     float64
 6   rent_perWeek    25 non-null     float64
 7   description     50 non-null     object 
 8   propertyType    50 non-null     object 
 9   location        50 non-null     object 
 10  agent           50 non-null     object 
 11  listing_source  50 non-null     object 
 12  listing_url     50 non-null     object 
 13  listed_date     50 non-null     object 
dtypes: float64(5), object(9)
memory usage: 5.6+ KB


In [99]:
omt_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46 entries, 0 to 45
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   transaction     46 non-null     object 
 1   address         44 non-null     object 
 2   bedroom         44 non-null     object 
 3   bathroom        43 non-null     float64
 4   sales_price     21 non-null     float64
 5   rent_perMonth   23 non-null     float64
 6   rent_perWeek    23 non-null     float64
 7   description     44 non-null     object 
 8   propertyType    40 non-null     object 
 9   location        44 non-null     object 
 10  agent           4 non-null      object 
 11  listing_source  46 non-null     object 
 12  listing_url     46 non-null     object 
 13  listed_date     44 non-null     object 
dtypes: float64(4), object(10)
memory usage: 5.2+ KB


In [100]:
z_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   transaction     50 non-null     object 
 1   address         50 non-null     object 
 2   bedroom         44 non-null     float64
 3   bathroom        41 non-null     float64
 4   living_room     30 non-null     float64
 5   sales_price     25 non-null     float64
 6   rent_perMonth   25 non-null     float64
 7   rent_perWeek    25 non-null     float64
 8   description     50 non-null     object 
 9   propertyType    46 non-null     object 
 10  location        50 non-null     object 
 11  agent           50 non-null     object 
 12  listing_source  50 non-null     object 
 13  listing_url     50 non-null     object 
 14  listed_date     50 non-null     object 
dtypes: float64(6), object(9)
memory usage: 6.0+ KB


### Create a Unique ID for each data

The unique id will be crated by combining the  [[transaction,agent,address,sales_price,rent_permonth,rent_perweek]] as one attribute and drop all empty columns

In [101]:
# create a unique id field
rm_data['id'] = rm_data['transaction'] + rm_data["bedroom"].astype(str) + rm_data["bathroom"].astype(str) + rm_data["sales_price"].astype(str) + z_data["rent_perMonth"].astype(str) + rm_data["address"]
# drop empty columns
rm_data.dropna(subset=['address'], inplace=True)

# change the data column to data type
rm_data['listed_date'] = pd.to_datetime(rm_data['listed_date'])

# read dataset
rm_data.head()

,transaction,address,bedroom,bathroom,sales_price,rent_perMonth,rent_perWeek,description,propertyType,location,agent,listing_source,listing_url,listed_date,id
0,rent,Narcissus Road London NW6,1.0,1.0,NaN,1750.0,404.0,A large and modern one bedroom apartment boast...,Flat,NW6,"Kinleigh Folkard & Hayward - Lettings, West Ha...",rightmove,https://www.rightmove.co.uk/properties/1348085...,2023-05-15,rent1.01.0nanNarcissus Road London NW6
1,rent,"Flat 2, Refectory Apartments, 3 Lawrence Yard,...",2.0,1.0,NaN,1750.0,404.0,Ellis & Co are delighted to offer on the marke...,Apartment,N15,"Ellis & Co, Tottenham",rightmove,https://www.rightmove.co.uk/properties/1350452...,2023-05-19,"rent2.01.0nanFlat 2, Refectory Apartments, 3 L..."
2,rent,"Essex Road, Islington, N1",3.0,1.0,NaN,3250.0,750.0,This superbly located three double bedroom spl...,Apartment,N1,"Charles Henry Peppiatt Ltd, Southgate",rightmove,https://www.rightmove.co.uk/properties/1350452...,2023-05-19,"rent3.01.0nanEssex Road, Islington, N1"
3,rent,"Kingston Road, London, SW20",1.0,1.0,NaN,1450.0,335.0,"Modern 1 bed, 1 bath flat located within walki...",Flat,SW20,"Criterion Hospitality Limited, Criterion Hospi...",rightmove,https://www.rightmove.co.uk/properties/1350058...,2023-05-19,"rent1.01.0nanKingston Road, London, SW20"
4,rent,"Adelaide Road, Chalk Farm, NW3",1.0,1.0,NaN,1650.0,381.0,This charming one bedroom apartment is on the ...,Apartment,NW3,"Charles Henry Peppiatt Ltd, Southgate",rightmove,https://www.rightmove.co.uk/properties/1335260...,2023-05-19,"rent1.01.0nanAdelaide Road, Chalk Farm, NW3"


In [104]:
# create a unique id field
omt_data['id'] = omt_data['id'] = omt_data['transaction'] + omt_data["bedroom"].astype(str) + omt_data["bathroom"].astype(str) + omt_data["sales_price"].astype(str) + z_data["rent_perMonth"].astype(str) + omt_data["address"]
# drop empty columns
omt_data.dropna(subset=['address'], inplace=True)

# # change the data column to data type
# omt_data['listed_date'] = pd.to_datetime(omt_data['listed_date'])

# read dataset
omt_data.head()

,transaction,address,bedroom,bathroom,sales_price,rent_perMonth,rent_perWeek,description,propertyType,location,agent,listing_source,listing_url,listed_date,id
0,rent,Albion Street London W2,2,2.0,NaN,6500.0,1500.0,2 bedroom apartment to rent,apartment,W2,Kinleigh Folkard & Hayward - Bayswater Lettings,omt,https://www.onthemarket.com/details/13180679/,< 14 days,rent22.0nanAlbion Street London W2
1,rent,"Provost Way, Dagenham, London,, Dagenham, Lond...",1,1.0,NaN,900.0,208.0,1 bedroom in a house share to rent,house,RM8,Home World Management - London,omt,https://www.onthemarket.com/details/13216612/,< 7 days,"rent11.0nanProvost Way, Dagenham, London,, Dag..."
2,rent,"Chalklands, Wembley HA9",1,1.0,NaN,750.0,173.0,1 bedroom flat to rent,flat,HA9,NaN,omt,https://www.onthemarket.com/details/13215915/,< 7 days,"rent11.0nanChalklands, Wembley HA9"
3,rent,"Elmhurst Road, Enfield, London, Enfield EN3",1,1.0,NaN,1000.0,231.0,1 bedroom flat to rent,flat,EN3,NaN,omt,https://www.onthemarket.com/details/13215647/,< 7 days,"rent11.0nanElmhurst Road, Enfield, London, Enf..."
4,rent,"Hogarth Road , Kensington , Kensington Chelsea...",1,1.0,NaN,1647.0,380.0,1 bedroom flat to rent,flat,SW5,NaN,omt,https://www.onthemarket.com/details/8560268/,< 7 days,"rent11.0nanHogarth Road , Kensington , Kensing..."


In [115]:
# create a unique id field
z_data['id'] = z_data['id'] = z_data['transaction'] + z_data["bedroom"].astype(str) + z_data["bathroom"].astype(str) + z_data["sales_price"].astype(str) + z_data["rent_perMonth"].astype(str) + z_data["address"]
# drop empty columns
z_data.dropna(subset=['address'], inplace=True)

# change the data column to data type
z_data['listed_date'] = pd.to_datetime(z_data['listed_date'])

# read dataset
z_data.head()

,transaction,address,bedroom,bathroom,living_room,sales_price,rent_perMonth,rent_perWeek,description,propertyType,location,agent,listing_source,listing_url,listed_date,id
0,rent,"Belvedere Heights, Lisson Grove, London NW8",2.0,2.0,1.0,NaN,2300.0,531.0,This is a beautifully decorated modern two dou...,flat,NW8,Redac Strattons - Central London Hub,zoopla,https://www.zoopla.co.uk/to-rent/details/64672...,2023-05-19,"rent2.02.0nan2300.0Belvedere Heights, Lisson G..."
1,rent,"Narcissus Road, West Hampstead, London NW6",1.0,1.0,NaN,NaN,1500.0,346.0,All bills included! Introducing a stunning new...,Studio,NW6,Belgrave Estates,zoopla,https://www.zoopla.co.uk/to-rent/details/64672...,2023-05-19,"rent1.01.0nan1500.0Narcissus Road, West Hampst..."
2,rent,"Hoxton Street, London N1",2.0,1.0,NaN,NaN,2496.0,576.0,"The Flat is situated on Hoxton Street, which i...",flat,N1,MK London Properties,zoopla,https://www.zoopla.co.uk/to-rent/details/64672...,2023-05-19,"rent2.01.0nan2496.0Hoxton Street, London N1"
3,rent,"Cambridge Rd, Kingston Upon Thames, London KT1",1.0,1.0,NaN,NaN,1600.0,369.0,Material Information Council Tax Band :C,flat,KT1,B & K Estates,zoopla,https://www.zoopla.co.uk/to-rent/details/64672...,2023-05-19,"rent1.01.0nan1600.0Cambridge Rd, Kingston Upon..."
4,rent,"Old Oak Common Lane, East Acton, London W3",1.0,NaN,NaN,NaN,750.0,173.0,Savoy Property Consultants are delighted to pr...,NaN,W3,Savoy Property Consultants,zoopla,https://www.zoopla.co.uk/to-rent/details/54451...,2023-05-19,"rent1.0nannan750.0Old Oak Common Lane, East Ac..."


### Connecting to the database

In [ ]:

def create_database(database_name: str):
    # Connect to the default database
    conn = psycopg2.connect("host=localhost dbname=postgres user=postgres password=1118")
    conn.set_session(autocommit=True)
    cur = conn.cursor()

    # Create the sparkify database with UTF8 encoding
    cur.execute(f"DROP DATABASE IF EXISTS {database_name}")
    cur.execute(f"CREATE DATABASE {database_name} ENCODING 'UTF8'")

    # Close the connection to the default database
    cur.close()
    conn.close()

    # Connect to the new database
    conn = psycopg2.connect(f"host=localhost dbname={database_name} user=postgres password=1118")
    conn.set_session(autocommit=True)
    cur = conn.cursor()

    return cur, conn

cur, conn = create_database('london_propertylisting')

### Create table for each dataset and insert the data into the table
float is used instead of integer because there are NAN values which not be represented as integer

Right move

In [107]:
print(pd.io.sql.get_schema(rm_data, name="rm_data"))

CREATE TABLE "rm_data" (
"transaction" TEXT,
  "address" TEXT,
  "bedroom" REAL,
  "bathroom" REAL,
  "sales_price" REAL,
  "rent_perMonth" REAL,
  "rent_perWeek" REAL,
  "description" TEXT,
  "propertyType" TEXT,
  "location" TEXT,
  "agent" TEXT,
  "listing_source" TEXT,
  "listing_url" TEXT,
  "listed_date" TIMESTAMP,
  "id" TEXT
)


In [119]:
cur.execute("DROP TABLE IF EXISTS rightmove")

rm_table = ("""CREATE TABLE IF NOT EXISTS rightmove (
transaction TEXT,
  address VARCHAR,
  bedroom FLOAT,
  bathroom FLOAT,
  sales_price FLOAT,
  rent_perMonth FLOAT,
  rent_perWeek FLOAT,
  description VARCHAR,
  propertyType VARCHAR,
  location VARCHAR,
  agent VARCHAR,
  listing_source VARCHAR,
  listing_url VARCHAR,
  listed_date VARCHAR,
  id TEXT PRIMARY KEY
  );""")

cur.execute(rm_table)

In [120]:
# insert rm_data into the table created
rm_insert = """
INSERT INTO rightmove (
    transaction,
    address,
    bedroom,
    bathroom,
    sales_price,
    rent_perMonth,
    rent_perWeek,
    description,
    propertyType,
    location,
    agent,
    listing_source,
    listing_url,
    listed_date,
    id)
VALUES
(%s,%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
"""
for i, row in rm_data.iterrows():
    cur.execute(rm_insert, list(row))

On the market

In [130]:
cur.execute("DROP TABLE IF EXISTS omt")

omt_table = ("""CREATE TABLE IF NOT EXISTS omt (
transaction TEXT,
  address VARCHAR,
  bedroom FLOAT,
  bathroom FLOAT,
  sales_price FLOAT,
  rent_perMonth FLOAT,
  rent_perWeek FLOAT,
  description VARCHAR,
  propertyType VARCHAR,
  location VARCHAR,
  agent VARCHAR,
  listing_source VARCHAR,
  listing_url VARCHAR,
  listed_date VARCHAR,
  id TEXT PRIMARY KEY
  );""")

cur.execute(omt_table)

In [131]:
# insert omt_data into the table created
omt_insert = """
INSERT INTO omt (
    transaction,
    address,
    bedroom,
    bathroom,
    sales_price,
    rent_perMonth,
    rent_perWeek,
    description,
    propertyType,
    location,
    agent,
    listing_source,
    listing_url,
    listed_date,
    id)
VALUES
(%s,%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
"""
for i, row in omt_data.iterrows():
    cur.execute(omt_insert, list(row))

InvalidTextRepresentation: invalid input syntax for type double precision: "Studio"
LINE 19: ('rent','Browning Way, Hounslow TW5', 'Studio', 1.0, 'NaN'::...
                                               ^


Zoopla

In [123]:
z_table = ("""CREATE TABLE IF NOT EXISTS zoopla (
transaction TEXT,
  address VARCHAR,
  bedroom FLOAT,
  bathroom FLOAT,
  living_room FLOAT,
  sales_price FLOAT,
  rent_perMonth FLOAT,
  rent_perWeek FLOAT,
  description VARCHAR,
  propertyType VARCHAR,
  location VARCHAR,
  agent VARCHAR,
  listing_source VARCHAR,
  listing_url VARCHAR,
  listed_date VARCHAR,
  id TEXT PRIMARY KEY
  );""")

cur.execute(z_table)

In [124]:
# insert rm_data into the table created
z_insert = """
INSERT INTO zoopla (
    transaction,
    address,
    bedroom,
    bathroom,
    living_room,
    sales_price,
    rent_perMonth,
    rent_perWeek,
    description,
    propertyType,
    location,
    agent,
    listing_source,
    listing_url,
    listed_date,
    id)
VALUES
(%s,%s, %s, %s, %s, %s,%s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
"""
for i, row in z_data.iterrows():
    cur.execute(z_insert, list(row))

UniqueViolation: duplicate key value violates unique constraint "zoopla_pkey"
DETAIL:  Key (id)=(rent2.02.0nan2300.0Belvedere Heights, Lisson Grove, London NW8) already exists.


### Pull out the dataset and write query

  Use LEFT outer join to combine all the three dataset as one, remove duplicate and save the distinct data set into a new table in the database and also ouput it as csv

In [ ]:
query1 = """SELECT (name), COUNT(*) AS name_count
FROM user_info
GROUP BY name
ORDER BY name_count desc
LIMIT 10;"""

In [ ]:
cur.execute(query1)

for row in cur.fetchall():
    print(row)